Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a [kaggle competition](https://www.kaggle.com/c/whiskey-201911/) We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

We are going to run increasingly sophisticated classification models on our whisky reviews in parts 1, 2, and 3. For each of parts 1, 2, and 3, submit your best model's results to the Kaggle competition to measure `generalization accuracy` -- i.e. how well the model performs on new data.

##1. Classifier based on TfIdf vectorization of reviews

### Follow Along 

1. Join the Kaggle Competition
2. Download the data
3. Train and hyperparameter tune a model using an sklearn pipeline

### 1.0 Setup

#### 1.0.1 Get spacy and restart runtime

#### 1.0.2 import necessary packages, load spacy

In [1]:
import pandas as pd
import re

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import spacy

Load `spacy`

In [11]:
#YOUR CODE HERE
nlp = spacy.load("en_core_web_md", disable=['parser', 'tagger'])

#### 1.0.3 Load Kaggle Whisky Competition Data
The goal is to predict the rating from the review text

In [3]:
# !!!!! You may need to change the path !!!!!
# You can download these datasets from the Kaggle in-class 
# competition for your cohort. 
 
train = pd.read_csv('/Users/gregorysamek/github/DS-Unit-4-Sprint-1-NLP/module3-document-classification/Kaggle Data/train.csv')
test = pd.read_csv('/Users/gregorysamek/github/DS-Unit-4-Sprint-1-NLP/module3-document-classification/Kaggle Data/test.csv')

In [4]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### 1.1 Clean Text

In [7]:
def clean_doc(text):
  # COMPLETE THE CODE IN THIS CELL
  # remove new line characters
  text = text.replace('\\n', ' ')
  # remove numbers from the text
  text = re.sub('[^a-zA-Z]', ' ', text)
  # remove multiple white spaces
  text = re.sub(r'\s+', ' ', text)

  # case normalize and strip extra white spaces on the far left and right hand side
  text = text.lower().strip()
  return text

train['description'] = train['description'].apply(clean_doc)
test['description'] = test['description'].apply(clean_doc)


In [8]:
train['description'][0]

'sometimes when whisky is batched a few leftover barrels are returned to the warehouse canadian club recently pulled and vatted several of these from the s acetone granny smith apples and fresh cut white cedar showcase this long age complex and spicy yet reserved this dram is ripe with strawberries canned pears cloves pepper and faint flowers then slightly pulling oak tannins distinct elegant and remarkably vibrant this ancient canadian club is anything but tired australia only a'

### 1.2 Split training data into Feature Matrix `X` and Target Vector `y`

In [10]:
target = 'category'
# COMPLETE THE CODE IN THIS CELL
X = train['description']
y = train['ratingCategory']

### 1.3 Specify the Model and Define the Pipeline Components

For the classifier model, you can try any or several of 
* `RandomForestClassifier()` or `GradientBoostingClassifier()` from the `sklearn` library
* `XGBClassifier()` from the `xgboost` library
* `CatboostClassifier()` from the `catboost` library
* `LGBMClassifier()` from the `lightgbm` library


In [12]:
# limit max_features to 500 to speed up training on Colab.
# COMPLETE THE CODE IN THIS CELL
vect = TfidfVectorizer(stop_words='english')
clf = XGBClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### 1.4 Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [24]:
# COMPLETE THE CODE IN THIS CELL
# Parameters to search in dictionary 
parameters = {
    'vect__max_df': (0.25, 0.3, 0.35),
    'clf__max_depth':(12, 14, 16)
}

# Implement a grid search with cross-validation
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Display the best score from the grid search
grid_search.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


0.7408905501372127

In [25]:
# Display the best parameters from the grid search
print(grid_search.best_params_)

{'clf__max_depth': 16, 'vect__max_df': 0.35}


### 1.5 Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so only submit when your predicted test accuracy is the highest you can make it. For this competition the max daily submissions are capped at **20**.  The submission file is made from the results of running your best model on the **test data set**, for which we don't get the targets.<br><br>

In [26]:
# COMPLETE THE CODE IN THIS CELL
# Predictions on **test** sample
pred = grid_search.predict(test['description'])

In [27]:
pred

array([1, 1, 1, ..., 1, 0, 0])

In [28]:
# COMPLETE THE CODE IN THIS CELL
submission = pd.DataFrame({'id' : test['id'], 'category': pred})
submission['category'] = submission['category'].astype('int64')

In [29]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission_number = 0

submission.to_csv(f'submission{submission_number}.csv', index=False)
submission_number += 1

In [ ]:
# Download submission to local machine from this Google Colab notebook
from google.colab import files
files.download(f'submission{submission_number}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 1.6 Submit your results to `kaggle` and get your score

First, upload the `kaggle.json` API token file from your local machine.<br>
Do this by clicking the file icon in the left sidebar, <br>
then clicking file icon with an up arrow inside it at the upper left, <br>
then navigating to and selecting the `kaggle.json` file in your local machine.<br>
`kaggle.json` is usually found in a folder called `.kaggle` on your local machine, <br>
if you have a Kaggle account. Note that there is no cost involved in <br>registering for a Kaggle account.<br><br>

Then: make a folder `/root/.kaggle` in this notebook,<br>
and copy your `kaggle.json` file into the `/root/.kaggle/` folder

In [ ]:
#!mkdir /root/.kaggle/
!mv /kaggle.json /root/.kaggle/ 
!chmod 600 /root/.kaggle/kaggle.json # to safeguard your privacy
!ls -l /root/.kaggle/

total 4
-rw------- 1 root root 64 Dec  2 02:12 kaggle.json


Now you can submit your predictions to Kaggle
directly from this Colab notebook!<br>


In [ ]:
!kaggle competitions submit whiskey-201911 -f submission1.csv -m "submission1"

100% 1.91k/1.91k [00:04<00:00, 469B/s]
Successfully submitted to Whiskey

You can check your score on the [Kaggle Whisky Competition website](https://www.kaggle.com/c/whiskey-201911/submissions), 
<br>on the "My Submissions" tab:

_This submission got a score of $0.94186$_

## Challenge

You're trying to achieve a minimum of 75% Accuracy on your model.

## 2. Add Latent Semantic Indexing to your pipeline (Learn)
<a id="p2"></a>

### Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (LSI) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### 2.1 Define Pipeline Components

Nest pipelines to perform SVD on our vectorization (LSA)

In [30]:
# COMPLETE THE CODE IN THIS CELL
# Transforming our Vectorization with SVD is how LSA generates topic columns
svd = TruncatedSVD(n_components=2, 
                   algorithm='randomized', 
                   n_iter=10)

# vectorizer and classifier like before
vect = TfidfVectorizer(stop_words='english')
clf = XGBClassifier()

# LSA pipeline with vectorizer & truncated SVD
lsa = Pipeline([('vect', vect), ('svd', svd)])

# combine LSA pipeline together with classifier
pipe = Pipeline([('lsa', lsa), ('clf', clf)])

### 2.2 Define Your grid search space and run a grid search with cross-validation
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [32]:
# COMPLETE THE CODE IN THIS CELL
parameters = {
    'lsa__svd__n_components': (100, 500, 1000),
    'lsa__vect__max_df': (0.2, 0.4),
    'clf__max_depth': (12, 16)
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/Users/gregorysamek/anaconda3/envs/U4-S1-NLP/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('lsa',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD(n_iter=10))])),
                                       ('clf',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric=None,...
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'clf__max_depth': (12, 16),
                         'lsa__svd__n_components': (100, 500, 1000),
                         'lsa__vect__max_df': (0.2, 0.4)},
             verbose=1)

In [33]:
grid_search.best_score_

0.7377048267817852

In [34]:
grid_search.best_params_

{'clf__max_depth': 12,
 'lsa__svd__n_components': 1000,
 'lsa__vect__max_df': 0.2}

### 2.3 Make a Submission File
See section $1.6$ above for instructions on how to submit your results file to `kaggle` and get your score

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission{submission_number}.csv', index=False)
submission_number +=1

In [ ]:
# Download submission to your local machine from this Colab notebook
from google.colab import files
files.download(f'submission{submission_number}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# 3. Add Spacy Word Embeddings
<a id="p3"></a>

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use document vectors made from those word embeddings as your features for a classification model.
4. Make a submission to Kaggle 

### 3.1 Process the data set with spacy

In [35]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [38]:
# Continue Word Embedding Work Here
nlp = spacy.load("en_core_web_md")

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

X_train_emb = get_word_vectors(train['description'])
X_test_emb = get_word_vectors(test['description'])

In [39]:
rfc = RandomForestClassifier(oob_score=True)

rfc.fit(X_train_emb, y)

RandomForestClassifier(oob_score=True)

In [40]:
# massively overfit with the Random Forest
print('Training Accuracy: ', rfc.score(X_train_emb, y))

Training Accuracy:  0.9997553217518963


Here we use oob_score_ (out-of-bag score) as a **proxy** for the test score;<br>
for your submission, you will predict on the test set, as before

In [41]:
# validation looks decent without any tuning

rfc.oob_score_

0.7200880841693174

### 3.2 Make a Submission File
See section $1.6$ above for instructions on how to submit your results file to `kaggle` and get your score

### Make a Submission File

In [ ]:
# YOUR CODE HERE

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission_number = 2
submission.to_csv(f'submission{submission_number}.csv', index=False)

In [ ]:
# Download submission to local machine from Google Colab
from google.colab import files
files.download(f'submission{submission_number}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 3.3 Submit your predictions to Kaggle


---



In [ ]:
# YOUR CODE HERE 

# Post Lecture Assignment (Stretch)
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. <br>
Once you've accomplished that, do (1), and either (2) or (3): 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do people create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?

2. Singular Value Decomposition (SVD) is one of the most important and powerful methods in Applied Mathematics and in all of Machine Learning.  Principal Components Analysis (PCA) -- which we used in Module 2 -- is closely releated to SVD. Research SVD using the resources below. Then write a few paragraphs explaining -- in your own words -- your understanding of SVD and why it has become so important in Machine Learning. As you write, pretend that you will be presenting this summary orally as an answer to a question during a job interview.<br>

* [Daniela Witten](https://www.danielawitten.com/), a Professor of Mathematical Statistics at the University of Washington, recently penned a highly amusing and informative [tweetstorm](https://twitter.com/WomenInStat/status/1285611042446413824) about SVD, well worth reading!<br>
* [Stanford University Lecture on SVD](https://www.youtube.com/watch?v=P5mlg91as1c) <br>
* [StatQuest Principal Components Analysis](https://www.youtube.com/watch?v=FgakZw6K1QQ)<br>
* [Luis Serrano Principal Components Analysis](https://www.youtube.com/watch?v=g-Hb26agBFg)<br>

3. Research which other models can be used for text classification -- see [Multi-Class Text Classification Model Comparison and Selection](https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568)
  - Try a few other classical machine learning models, and compare with the gradient boosting results 
  - Neural Networks are becoming more popular for document classification. Why is that the case? 
  - If you have the time and interest, check out this [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google
   